# Part 1: Tabular Q-Learning

In this part of the assignment you will familiarize yourself with the [Gymnasium API](https://gymnasium.farama.org/) for reinforcement learning and implement a tabular Q-learning agent to learn a behavioral policy in the simple [Taxi environment](https://gymnasium.farama.org/environments/toy_text/taxi/). This is intended as a warmup on a relatively small and simple task, preparing for a more challenging deep reinforcement learning task in part 2.

**Learning objectives.** You will:
1. Install and study the standard Gymnasium API for representing a reinforcement learning environment simulation
2. Implement the tabular Q-learning algorithm with epsilon greedy action selection
3. Train an agent / compute an optimal q function and policy, experimenting with various hyperparameters like the learning rate and exploration decay
4. Evaluate the learning curve of average episodic return over training

We will use the [Gymnasium API](https://gymnasium.farama.org/) for our reinforcement learning environments. First you should run one of the following to install the necessary packages (either using `pip` or `conda` depending on your preferred Python package manager).

In [5]:
pip install "gymnasium[all]"

  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for swig==4.* from https://files.pythonhosted.org/packages/32/5a/fd3de7fa6ef45b8bb39723de65aeedbe7b213ee397d2e9fa9fefe0507636/swig-4.3.0-py2.py3-none-win_amd64.whl.metadata
  Using cached swig-4.3.0-py2.py3-none-win_amd64.whl.metadata (3.5 kB)
  Obtaining dependency information for mujoco-py<2.2,>=2.1 from https://files.pythonhosted.org/packages/37/e5/e7504cb2ded511910c2a2e8f9c9e28af075850eb03a5c5a8daee5d7d9517/mujoco_py-2.1.2.14-py3-none-any.whl.metadata
  Using cached mujoco_py-2.1.2.14-py3-none-any.whl.metadata (669 bytes)
  Obtaining dependency information for cython<3 from https://files.pythonhosted.org/packages/7e/26/9d8de10005fedb1eceabe713348d43bae1dbab1786042ca0751a2e2b0f8c/Cython-0.29.37-py2.py3-none-any.whl.metadata
  Using cached Cython-0.29.37-py2.py3-none-any.whl.metadata (3.1 kB)
  Obtaining

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [16 lines of output]
  Using setuptools (version 75.2.0).
  C:\Users\micha\anaconda3\Lib\site-packages\setuptools\_distutils\dist.py:261: UserWarning: Unknown distribution option: 'test_suite'
    warnings.warn(msg)
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-cpython-311\Box2D
  copying library\Box2D\Box2D.py -> build\lib.win-amd64-cpython-311\Box2D
  copying library\Box2D\__init__.py -> build\lib.win-amd64-cpython-311\Box2D
  creating build\lib.win-amd64-cpython-311\Box2D\b2
  copying library\Box2D\b2\__init__.py -> build\lib.win-amd64-cpython-311\Box2D\b2
  running build_ext
  building 'Box2D._Box2D' extension
  swigging Box2D\Box2D.i to Box2D\Box2D_wrap.cpp
  swig.exe -python -c++ -IBox2D -small -O -includeall -ignoremissing -w201 -globals b2Globals -outdir library\Box2D -keyword -w511 -D_SWIG_KWARGS -o Box2D\Box2D_wrap.cpp 

In [4]:
conda install -c conda-forge gymnasium-all


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.11.0




Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\micha\anaconda3

  added / updated specs:
    - gymnasium-all


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    absl-py-2.1.0              |     pyhd8ed1ab_0         105 KB  conda-forge
    ale-py-0.9.0               |  py311hde19ee9_0         914 KB  conda-forge
    aom-3.6.1                  |       h63175ca_0         7.6 MB  conda-forge
    box2d-py-2.3.8             |  py311hda3d55a_9         374 KB  conda-forge
    ca-certificates-2024.8.30  |       h56e8100_0         155 KB  conda-forge
    cairo-1.16.0               |    hd694305_1014         2

If you have trouble with the above installation, try the simpler `pip install gymnasium` or `conda install -c conda-forge gymnasium` command (which does not install all of the dependencies that may only be needed for specific environments). 

You can check that the installation was successful by running the following code. The code attempts to import the gymnasium module, create a simple [Taxi environment](https://gymnasium.farama.org/environments/toy_text/taxi/), and then execute random actions in the environment. 

The environment is set to `render_mode="human"`, meaning you should see a visualization of the environment with the agent taking random actions. Be aware that the visualization can be unstable on some platforms -- in particular, if your Python Kernel crashes after closing the visualization, that will not prevent you from completing the assignment (but is a good indication that you should save your learned Q tables/networks before rendering a visualization so that you don't lose any work).

In [ ]:
import gymnasium as gym

# Initialise the environment
env = gym.make("Taxi-v3", render_mode="human")

# Reset the environment to generate the first observation
observation, info = env.reset()
for _ in range(100):
    # this is where you would insert your policy
    action = env.action_space.sample()

    # step (transition) through the environment with the action
    # receiving the next observation, reward and if the episode has terminated or truncated
    observation, reward, terminated, truncated, info = env.step(action)

    # If the episode has ended then we can reset to start a new episode
    if terminated or truncated:
        observation, info = env.reset()

env.close()

Next you should review the [Gymnasium Basic Usage Documentation](https://gymnasium.farama.org/introduction/basic_usage/) (through the Action and observation spaces section; you do not need to worry about the section on Modifying the environment). It may also be helpful to review or refer to the [Environment API documentation here](https://gymnasium.farama.org/api/env/).

## Task 1

Your first task is to define a **tabular** Q-learning agent for the simple [Taxi Environment](https://gymnasium.farama.org/environments/toy_text/taxi/). You should refer to the environment documentation to understand the state/observation space, action space, and reward function.

Your agent `class` should implement at least the following three methods:
1. The constructor, `__init__()`, should should store any relevant hyperparameters as instance variables and initialize a Q table, along with any other auxiliary instance variables you may choose to include.
2. The `action_select` method which should implement `epsilon` greedy action selection (`epsilon` should be an instance variable kept track of by the agent object).
3. The `update` method should take an environment experience (previous state, action taken, reward received, new state, and whether the episode terminated) as input and should perform a Q learning update on the Q table.

 You are welcome but not required to add additional helper methods.

In [ ]:
class TaxiAgent:
    """
    Implements a tabular q learning agent for the taxi 
    environment with discrete states and actions.
    """
    def __init__(self, env, lr=0.01, discount=0.99,
                 epsilon_init=1.0, epsilon_final=0.05,
                 epsilon_decay=0.99):
        # todo: complete constructor/initialization
        pass # remove pass when implementing


    def action_select(self, s):
        # todo: complete epsilon greedy action selection
        pass # remove pass when implementing


    def update(self, s_old, a, r, s_new, terminated):
        # todo: complete Q learning update
        pass # remove pass when implementing

## Task 2

Now that you have defined your learning agent, implement a training loop over several episodes of experience with the environment. Your goal is to learn an optimal q function and corresponding policy (in the Taxi environment, that just implies efficiently picking up and delivering passengers to their appropriate destimations). 

Practically speaking, you should be able to learn a policy that achieves an average/expected episode return (cumulative reward) of approximately 0 (because the agent receives 20 reward for a correct dropoff, and -1 for each timestep, and it will require 10-20 timesteps on average to complete a pickup and dropoff). To accomplish this you may need to experiment with several of the key hyperparameters for the learning agent.

While you are not required, you are encouraged to print periodic evaluation information during training (for example, every 100 or so episodes you may wish to printout the average return of the last 100 episodes) in order to track progress.

Training should be quick on this environment: You should be able to learn an essentially optimal policy within several hundred or a few thousand episodes requiring only a few seconds of time.

In [ ]:
# Initialise the environment
import gymnasium as gym

# todo: adjust n_episodes as needed for training
# default value of 100 for prototyping, will likely
# want more episodes for training
n_episodes = 100

env = gym.make("Taxi-v3")

# wrapper saves episode information necessary for later evaluation in task 3
env = gym.wrappers.RecordEpisodeStatistics(env, buffer_length = n_episodes)

# todo: environment loop with Q learning

## Task 3

After training, you can use the `visualize_learning_curves` function below to create two plots: One of the moving average episode return over the course of training and another of the moving average episode length over the course of training. A `smoothing` parameter to the function determines how many episodes to average over. That is, the default of `smoothing=10` means that each point in the resulting figure is the average of ten episodes. This has the visual effect of *smoothing* out the appearance of the plot, which can make the overall trend easier to visualize even if individual episodes vary significantly.

Create visualizations of your learning curves. You should see improving performance on average episode return converging toward approximately zero.

In [ ]:
# Run, but you do not need to modify this code

import numpy as np
from matplotlib import pyplot as plt

def visualize_learning_curves(env, smoothing=10):
    fig, axs = plt.subplots(1, 2, figsize=(20, 8))

    axs[0].plot(np.convolve(env.return_queue, np.ones(smoothing), 'valid') / smoothing)
    axs[0].set_title("Episode Returns", fontsize=20)
    axs[0].set_xlabel("Episode", fontsize=20)
    axs[0].set_ylabel("Return", fontsize=20)

    axs[1].plot(np.convolve(env.length_queue, np.ones(smoothing), 'valid') / smoothing)
    axs[1].set_title("Episode Lengths", fontsize=20)
    axs[1].set_xlabel("Episode", fontsize=20)
    axs[1].set_ylabel("Length", fontsize=20)

    plt.tight_layout()
    plt.show()

In [ ]:
# todo: visualize learning curves

## Optional

The above evaluation of the learning curve was purely quantiative. While you are not required, you are invited and encouraged to qualitatively evaluate your learned policy/agent visually below. To do so, render the environment in `"human"` mode as shown below. This should render the visualization of the environment. Write a standard environment loop over several episodes (but far fewer than used for training, perhaps just 5 or so), and use the trained agent for action selection. For this evaluation purpose, you are welcome to set `epsilon` to a lower value or even 0, and you do not need to perform learning updates.

Again, be aware that the visualization can be unstable on some platforms -- in particular, if your Python Kernel crashes after closing the visualization, that will not prevent you from completing the assignment (but is a good indication that you should save your learned Q tables/networks before rendering a visualization so that you don't lose any work).

In [ ]:
# Initialise the environment
env = gym.make("Taxi-v3", render_mode="human")

# optional todo: environment loop to visualize learned policy